In [3]:
#imports
from rply import LexerGenerator

In [4]:
lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')
lg.add('ABREPARENTESES', r'\(')
lg.add('FECHAPARENTESES', r'\)')

lg.add('IGUAL', r'recebe')
lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('PONTOVIRGULA', r'\;')

#adicionar if
lg.add('SE', r'se')
lg.add('FAZ', r'faz')
lg.add('CASO_CONTRARIO', r'caso_contrario_faz')
lg.add('FIM_SE', r'fim_se')
lg.add('ENQUANTO', r'enquanto')
lg.add('PARA_CADA', r'para_cada')

lg.add('IDENTIFICADOR', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMPARADOR','for_igual_a')
lg.add('COMPARADOR','for_diferente_de')
lg.add('COMPARADOR','for_maior_ou_igual_a')
lg.add('COMPARADOR','for_menor_ou_igual_a')
lg.add('COMPARADOR','for_maior_que')
lg.add('COMPARADOR','for_menor_que')


lg.ignore('\s+')
lexer = lg.build()

In [6]:
tokens = lexer.lex("x gustavo y inteiro x recebe 10 y recebe 20")
for token in tokens:
    print(token)

Token('VARIAVEL', 'x')
Token('VARIAVEL', 'gustavo')
Token('VARIAVEL', 'y')
Token('INTEIRO', 'inteiro')
Token('VARIAVEL', 'x')
Token('IGUAL', 'recebe')
Token('NUMERO', '10')
Token('VARIAVEL', 'y')
Token('IGUAL', 'recebe')
Token('NUMERO', '20')
